In [2]:
import mido
import string
import numpy as np
import scipy
#mid = mido.MidiFile('.mid', clip=True)
mid = mido.MidiFile('test.mid', clip=True)
mid.tracks

[<midi track 'Piano\x00' 63 messages>]

In [3]:
def convert_clocks_per_click(clocks_per_click, user_tempo):
    sec_per_click = .6 / (user_tempo * clocks_per_click)
    return sec_per_click

In [4]:
def convert_velocity(midi_vel):
    velocity = midi_vel / 127.0
    return velocity

In [5]:
def convert_frequency(midi_val):
    reference = 440
    frequency = (reference / 32) * (2 ** ((midi_val - 9) / 12))
    return frequency

In [6]:
def parse_MIDI(midi_file, user_tempo):
    # ticks_per_quarter = <PPQ from the header>
    # µs_per_quarter = <Tempo in latest Set Tempo event>
    # µs_per_tick = µs_per_quarter / ticks_per_quarter
    # seconds_per_tick = µs_per_tick / 1.000.000
    # seconds = ticks * seconds_per_tick
    clocks_per_click = 0
    midi_info = []
    note_value = 0
    note_velocity = 0
    note_dur = 0
    sec_per_click = 0


    mid = mido.MidiFile(midi_file, clip=True)
    for i in range(0, 2):
        for m in mid.tracks[i][:]:
            print(m)
            m = str(m)
            if 'clocks_per_click' in m and clocks_per_click == 0:
                # midi.append(m)
                temp = m.partition("clocks_per_click=")[2]
                clocks_per_click = temp.partition(" ")[0]
                sec_per_click = convert_clocks_per_click(int(clocks_per_click), user_tempo)
            if 'note_on' in m:
                temp = m.partition("note=")[2]
                note_value = temp.partition(" ")[0]
                #convert to frequency
                note_value = convert_frequency(int(note_value))
                temp = m.partition("velocity=")[2]
                note_velocity = temp.partition(" ")[0]
                #convert to 0-1 amplitude
                note_velocity = convert_velocity(int(note_velocity))
            if 'note_off' in m:
                note_dur = m.partition("time=")[2]
                #convert to seconds
                note_dur = sec_per_click * int(note_dur)
                if note_dur != 0:
                    midi_info.append((note_value, note_velocity, note_dur))
                # print(midi_info)
    return midi_info

In [7]:
note_list = parse_MIDI('scale.mid', 60)

<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
<meta message set_tempo tempo=250000 time=0>
<meta message copyright text='Copyright © ' time=0>
<meta message time_signature numerator=10 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=38400>
<meta message time_signature numerator=6 denominator=4 clocks_per_click=72 notated_32nd_notes_per_beat=8 time=9600>
<meta message end_of_track time=1>
program_change channel=0 program=0 time=0
control_change channel=0 control=121 value=0 time=0
control_change channel=0 control=64 value=0 time=0
control_change channel=0 control=91 value=48 time=0
control_change channel=0 control=10 value=51 time=0
control_change channel=0 control=7 value=100 time=0
<meta message track_name name='Piano' time=0>
pitchwheel channel=0 pitch=0 time=0
<meta message lyrics text='1/1 ' time=0>
note_on channel=0 note=60 velocity=76 time=0
note_off

In [8]:
def genSine(frequency, duration, amplitude = 1, sampleRate = 48000, phase = 0):
    
    #creates a sine wave
    
    import numpy as np
    
    time = np.arange(0, duration, 1/sampleRate)
    return amplitude * np.sin((2*np.pi * frequency * time) + phase)

In [9]:
def adsr(x,a=.25,d=.25,s=.25,r=.25,fs=48000):
    total_len = len(x)
    a_len = int(.25 * len(x))
    d_len = int(.25 * len(x))
    s_len = int(.25 * len(x))
    r_len = int(.25 * len(x))

    xa= np.arange(0, a_len)                                                     #sets up size of attack portion
    ya= np.linspace(0,np.max(x),xa.size)                                       #creates attack envelope

    xd= np.arange(a_len, a_len+d_len)                                                 #sets up size of decay portion
    yd= np.linspace(np.max(x),.5,xd.size)                                        #creates attack envelope

    xs= np.arange(a_len+d_len, a_len+d_len+s_len)                               #sets up size of sustain portion
    ys= np.linspace(.5,.5,xs.size)                                            #creates sustain envelope

    xr= np.arange(a_len+d_len+s_len, a_len+d_len+s_len+r_len)               #sets up size of release portion
    yr= np.linspace(.5,0,xr.size)

    env=np.concatenate((ya,yd,ys,yr))                                          #creates full adsr envelope array
    adsrnote=x*env[0:x.size]               
                                               #applies adsr envelope to input array
    return adsrnote

In [10]:
def lfo(x, freq, amplitude=1, fs=48000):
    t = np.arange(0,len(x)/fs,1/fs)
    lfo = x * amplitude * np.cos(2*np.pi*freq*t)
    return lfo

In [11]:
def reverb(x, impulse):
    fft_sig = scipy.fft.fft(x)
    fft_impulse = scipy.fft.fft(impulse)
    conv = fft_impulse * fft_sig
    return scipy.fft.ifft(conv)
    
    # from scipy.io.wavfile import read

    # fs, impulse = read('St Nicolaes Church.wav')
    # impulse = np.transpose(impulse)[0]

    # impulse = np.concatenate((impulse, np.zeros(len(play_list)-len(impulse))))
    # impulse = impulse / np.max(impulse)
    # play_list = np.real(reverb(play_list, impulse))


In [12]:
#function that applies a low-pass filter
#x is the original signal
#cutoff is the cutoff frequency
#order is the order of the filter
def lowpass(x, cutoff=440, order=10, fs=48000):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq

    from scipy.signal import butter, filtfilt

    (b,a) = butter(order, normal_cutoff, btype = 'low', analog = False)
    filt_sig = filtfilt(b,a,x)
    
    return filt_sig

In [27]:
def flanger(x, delay_time=0.003, rate=.1, feedback_percent, fs=48000):
    import math
    ind = np.arange(0, len(x))
    sine_osc = np.sin(2 * np.pi * ind * (rate/fs))
    delay_samples = int(delay_time*fs)
    x_zero = np.zeros(len(x))
    x_zero[:delay_samples] = x[:delay_samples]
    for i in range(delay_samples, len(x)):
        abs_sin = abs(sine_osc[i])
        cur_delay = math.ceil(abs_sin*delay_samples)
        x_zero[i] = (feedback_percent * x[i]) + (feedback_percent * x[i-cur_delay])
    return x_zero

In [82]:
def wahwah(x, damping=0.05, min_freq=250, max_freq=5000, wah_freq=1000, fs=48000):
    freq_change = wah_freq/fs
    center_freq = np.arange(min_freq, max_freq, freq_change)
    #create triangle wave of center frequencies
    while len(center_freq) < len(x):
        center_freq = center_freq[np.arange(max_freq,min_freq,-freq_change)]
        center_freq = center_freq[np.arange(min_freq,max_freq,freq_change)]
    center_freq = center_freq[0:len(x)]
    # center_freq = np.ndarray.transpose(center_freq)
    print(center_freq)

    freq_coeff = 2 * np.sin((np.pi * center_freq[0]) / fs)
    # print(center_freq)
    q = 2 * damping

    y_high = np.zeros(len(x))
    y_band = np.zeros(len(x))
    y_low = np.zeros(len(x))

    y_high[0] = x[0]
    y_band[0] = freq_coeff * y_high[0]
    y_low[0] = freq_coeff * y_band[0]

    for i in range(1, (len(center_freq))-1):
        y_high[i] = x[i] - y_low[i-1] - (q * y_band[i-1])
        y_band[i] = (freq_coeff * y_high[i]) + y_band[i-1]
        y_low[i] = (freq_coeff * y_band[i]) + y_low[i-1]
        freq_coeff = 2 * np.sin((np.pi * center_freq[i]) / fs)
    
    y_band = y_band / (max(abs(y_band)))
    return y_band


In [162]:
def wavetable(frequency, dur, fs=48000):
    from matplotlib import pyplot as plt
    # choose the wave type
    t = np.arange(0,128/fs,1/fs)
    wave = np.sin(2 * np.pi * 375 * np.arange(0,128/fs,1/fs))
    plt.plot(t, wave)
    wave = wave / max(wave)
    # frequency to step size
    step = (frequency * 128) / fs
    # time to desired sample length (array length)
    length = dur * fs
    ind_arr = np.arange(0, length, step)
    out = np.array([])
    #read table
    for i in ind_arr:
        i = i % 128
        if i != int(i):
            floored = int(i)
            next = floored + 1
            decimal = i - floored
            out = np.append(out, wave[floored] + decimal * (wave[next] - wave[floored]))
        elif i == int(i):
            out = np.append(out, wave[int(i)])
    return out


In [159]:
128.5 % 128

0.5

In [165]:
sine = wavetable(187.5, 1)

IndexError: index 128 is out of bounds for axis 0 with size 128

In [164]:
from IPython.display import Audio
import scipy
from scipy.io.wavfile import write
write("test.wav", 48000, sine)

In [119]:
print(sine)

[ 0.00000000e+00 -2.87383782e-16 -5.74767564e-16 ...  1.70218589e-14
 -8.27662913e-15 -3.35751172e-14]


In [102]:
int(3.9)

3

In [23]:
np.arange(0, 5, 2)

array([0, 2, 4])

In [25]:
note_list

[(261.6255653005986, 0.5984251968503937, 1.5999999999999999),
 (293.66476791740763, 0.6062992125984252, 1.5999999999999999),
 (311.1269837220809, 0.6062992125984252, 1.5999999999999999),
 (329.62755691286986, 0.6062992125984252, 1.5999999999999999),
 (349.22823143300394, 0.6062992125984252, 1.5999999999999999),
 (369.9944227116344, 0.6062992125984252, 1.5999999999999999),
 (391.9954359817492, 0.6062992125984252, 1.5999999999999999),
 (440.0, 0.6062992125984252, 1.5999999999999999),
 (466.1637615180898, 0.6062992125984252, 1.5999999999999999),
 (493.88330125612424, 0.6062992125984252, 1.5999999999999999),
 (523.2511306011972, 0.6062992125984252, 2.4),
 (523.2511306011972, 0.5984251968503937, 2.4)]

In [84]:
play_list = np.array([])
for i in note_list:
    note_sine = genSine(i[0], i[2], i[1])
    note_sine = adsr(note_sine)
    play_list = np.concatenate((play_list, note_sine))
    # play_list = lfo(play_list, 2)
    # play_list = lowpass(play_list, cutoff=440)
    # play_list = flanger(play_list, .004, 0.11, 0.57)
    # play_list = wahwah(play_list)
print(play_list)

[ 250.          250.02267574  250.04535147 ... 1849.93197279 1849.95464853
 1849.97732426]
[ 250.          250.02267574  250.04535147 ... 3449.93197279 3449.95464853
 3449.97732426]


IndexError: arrays used as indices must be of integer (or boolean) type

In [36]:
t = np.arange(0, 2, 1/44100)
np.cos(2*np.pi*200*t)

array([1.        , 0.99959404, 0.99837649, ..., 0.99634834, 0.99837649,
       0.99959404])

In [75]:
from IPython.display import Audio
import scipy
from scipy.io.wavfile import write
write("test.wav", 44100, play_list)

In [13]:
convert_clocks_per_click(36, 60)

0.0002777777777777778

In [14]:
convert_velocity(120)

0.9448818897637795

In [15]:
convert_frequency(69)

440.0